In [ ]:
###ATAC分析的作用
# 细胞类型鉴定:细胞的开放染色质区域、细胞的功能和特征。
# 发育过程研究:染色质可及性变化，揭示细胞的转录因子调控网络和细胞命运决定的机制。
# 细胞状态转换:研究细胞的分化过程、干细胞的自我更新和细胞的再编程过程等。
# 研究疾病和肿瘤异质性:疾病和肿瘤中细胞的染色质可及性变化、不同克隆细胞的异质性。
# 药物筛选和基因调控网络研究:研究药物对细胞染色质可及性的影响.从而发现新的治疗靶点和药物。

#其实ATAC测得是染色质的开放区域，利用Tn5转录酶进行捕获

In [ ]:
#为了进行这种细胞类型注释，使用细胞类型特异性标记基因，并通过基因评分从染色质可达性数
据中估计这些基因的基因表达。基因评分本质上是基于基因附近调控元素的可获得性，对基因表
达水平的预测。

In [ ]:
#第一步 
#先创建一个sample的文件，里面第一列是file.gz文件的绝对路径，第二列是规定的文件名字,列名就是file和sample


In [ ]:
import snapatac2 as snap
import numpy as np

snap.__version__

In [ ]:
0Ortshapatac2aSsnapLmportanndata as ad.mportaraparse
importpandas as pdmatplotlib.pyplot as pltimport
importscanpy as scLmport numpy as npparse=argparse.ArgumentParser(description='SnapATAC2')
parse.add argument!'.-fragmentlist’,help='the fragmentlist of fragment file name,loomfile eg:fragment, samplename',type str,required = True)
parse.add arqument('--outdir',help='the analysis dir',type=str)
parse.add argument(' -species',help='human or mouse',type=str,default = 'human')
argvparse.parse_args()

In [ ]:
sample =  pd.read_csv(fragmentlist,sep='.')

In [ ]:
if species == 'human' :
    adatas = snap.pp.import_data(list(sample['file']),file=[name + '.h5ad' for name in list(sample['sample'])],chrom_sizes=snap.genome.hg38,min_num_fragments=1000,sorted_by_barcode=False)
    snap.metrics.tsse(adatas, snap.genome.hg38)
else:
    adatas = snap.pp.import_data(list(sample['file']),file=[name + '.h5ad' for name in list(sample['sample'])],chrom_sizes=snap.genome.mm10,min_num_fragments=1000,sorted_by_barcode=False)
    snap.metrics.tsse(adatas, snap.genome.mm10)

In [ ]:
snap.metrics.tsse(adatas, snap.genome.hg38)
snap.pp.filter_cells(adatas, min_tsse=7)
snap.pp.add_tile_matrix(adatas, bin_size=5000)
snap.pp.select_features(adatas, n_features=50000)
#以下两步比较限速
snap.pp.scrublet(adatas)
snap.pp.filter_doublets(adatas)

In [ ]:
# ####Creting AnnDataSet object
data = snap.AnnDataSet(
    adatas=[(name, adata) for name, adata in zip(sample['sample'], adatas)],
    filename="colon.h5ads"
)
data

print(f'Number of cells: {data.n_obs}')
print(f'Number of unique barcodes: {np.unique(data.obs_names).size}')

In [ ]:
nique_cell_ids = [sa + ':' + bc for sa, bc in zip(data.obs['sample'], data.obs_names)]
data.obs_names = unique_cell_ids
assert data.n_obs == np.unique(data.obs_names).size

In [ ]:
snap.pp.select_features(data, n_features=50000)

In [ ]:
snap.tl.spectral(data)

In [ ]:
snap.tl.umap(data)
snap.pl.umap(data, color="sample", interactive=False)

In [ ]:
# Batch correction
snap.pp.mnc_correct(data, batch="sample")
snap.pp.harmony(data, batch="sample", max_iter_harmony=20)
snap.tl.umap(data, use_rep="X_spectral_mnn")
snap.pl.umap(data, color="sample", interactive=False)
plt.savefig(outdir + '/combined.atac.umap.sample.png',dpi= 300, bbox_inches='tight')

In [ ]:
#Clustering
snap.pp.knn(data, use_rep="X_spectral_harmony")
snap.tl.leiden(data)
snap.pl.umap(data, color="leiden", interactive=False)
plt.savefig(outdir + '/combined.atac.umap.leiden.png',dpi= 300, bbox_inches='tight')

In [ ]:
### cell cluster annotation 此处是活性矩阵
if species == 'human' :
    gene_matrix = snap.pp.make_gene_matrix(data, snap.genome.hg38)
else:
    gene_matrix = snap.pp.make_gene_matrix(data, snap.genome.mm10)

In [ ]:
###Imputation 插补
sc.pp.filter_genes(gene_matrix, min_cells=5)
sc.pp.normalize_total(gene_matrix)
sc.pp.log1p(gene_matrix)
gene_matrix.obsm['X_umap'] = data.obsm['X_umap']

In [ ]:
#Peak calling

snap.tl.macs3(data, groupby='leiden', replicate='sample')
if species == 'human' :
    merged_peaks = snap.tl.merge_peaks(data.uns['macs3'], chrom_sizes=snap.genome.hg38)
else:
    merged_peaks = snap.tl.merge_peaks(data.uns['macs3'], chrom_sizes=snap.genome.mm10)
merged_peaks.shape
peaks.to_csv(outdir + '/combined.peak.xls', sep= '\t')
peak_mat = snap.pp.make_peak_matirx(data, use_rep = peaks['Peaks'])

In [ ]:
### Finding marking regions 其实寻找开放区域的差异 细胞大类没有什么必要，一般都是细胞小类
marker_peaks = snap.tl.marker_regions(peak_mat, group_by = 'leiden', pvalue= 0.01)
snap.pl.regions(peak_mat, groupby='cell_type', peaks=marker_peaks, interactive=False)
plt.savefig(outdir + '/combined.atac.region.png',dpi = 300,bbox_inches = 'tight')

In [ ]:
#motif 看一下转录因子的调节活性 
if species == 'human':
    motifs = snap.tl.motif_enrichment(
        motifs=snap.datasets.cis_bp(unique=True),
        regions=marker_peaks,
        genome_fasta=snap.genome.hg38,)
else:
    motifs = snap.tl.motif_enrichment(
    motifs=snap.datasets.cis_bp(unique=True),
    regions=marker_peaks,
    genome_fasta=snap.genome.mm10,
)

snap.pl.motif_enrichment(motifs, max_fdr=0.0001, height=1600, interactive=False)
plt.savefig(outdir + '/combined.atac.motif.png',dpi =300, bbox_inches = 'tight')


In [ ]:
##RNA+ ATAC进行整合

In [ ]:
import snapatac2 as snap

In [ ]:
sample = pd.read_csv(samplist,sep = '.')
samples={}
for i in rang(sample.shape[0]):
    key = sample.iloc[i,0]
    samples[key] = sample.iloc[i,1]
adatas ={}
for sample_id, filename in samples.items():
    sample_adata = sc.read_10x_h5(filename)
    sample_adata.var_names_make_unique()
    adatas[sample_id]= sample_adata

In [ ]:
#### 整合
adata = ad.concat(adatas, label= 'sample')
adata.obs_names_make_unique()
sc.pp.filter_cells(adata, min_genes=100)sc.pp.filter_genes(adata, min_cells=5)
sc.pp.scrublet(adata, batch_key="sample"adata.layers["counts"]= adata.X.copy( )
sc.pp.normalize_total(adata)
sc.pp.loglp(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample"
sc.pp.scale(adata,max value=10)
sc.tl.pca(adata)
sce.pp.harmony_integrate(adata,'sample')
sc.pp.neighbors(adata,n pcs=30,n neighbors=30, random state=0,use_rep =['X_harmony']
sc.tl.leiden(adata)

In [ ]:
H##ATAC分
species谳争鲼亁艏
adatas = snap.pp.import data(list(sample[ ATC ]),file=[nameiby barcode=False)
semple'])],chrom sizes=snap.genome.hg38,min num fragments=1000,sort
snap.metrics.tsse(adatas, snap.genome.hg38)
lse:
adatas = snap.pp,import data(list(sample[ ATAc ]),file=[name +dby_barcode=False)
for name in list(sample[ sample'])],chrom_sizes=snap.genome.mm10,min_num_fragments=1000,sort
snap.metrics.tsse(adatas, snap.genome .mm10)
inap.pp.filter cells(adatas, min tsse=7)
snap.pp.add_tile_matrix(adatas, bin_size=5000)
snap.pp.select_features(adatas, n_features=50000)
inap.pp,scrublet(adatas)
snap.pp.filter doublets(adatas)



In [ ]:
##Creating AnnDataset object
atac = snap.AnnDataSet(adatas=[(name, adata) for name,adata in zip(sample['sample'], adatas)],filename="combined.h5ads")
unique cell ids =[sa +':'+ bc for sa, bc in zip(atac.obs[ sample ], atac.obs_names)]
atac.obs_names = unique_cell ids
assert atac.n_obs == np.unique(atac.obs_names).size
snap.pp.select features(atac,n features=50000)
snap.tl.spectral(atac)
###Batch correction
snap.pp.harmony(atac.batch="sample",max_iter_harmony=20)
snap,tl.umap(atac, use_rep="X_spectral_harmony")
####Performioint embedding
assert(adata.obs names == atac.obs names).all()
embedding = snap.tl.multi spectral([adata, atac], features=None)[1]
atac,obsm['x joint'] = embedding
snap.tl.umap(atac, use_rep='X joint')
snap.pl.umap(atac, color="cell type",interactive=False,height=550)
plt.savefig(outdir + '/combined,rna.atac,png',dpi = 300,bbox inches='tight')
atac.write(outdir + '/combined.atac.h5ad')

In [ ]:
#其实ATAC分析分两种
# 第一个就是一个样本做两个，分别是单细胞和ATAC，那么就可以将单细胞注释的结果直接套用在ATAC的数据上，因为一种细胞
# 第二个就是不用样本分别做了单细胞和ATAC，那么就需要进行整合